# **Compliance GPT with LangChain**

## **Setup**

In [26]:
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()

True

## **Config**

In [27]:
from utils.config import get_config

config = get_config()

In [28]:
USER_ID = 'xmriz'
CONVERSATION_ID = 'xmriz-2021-07-01-01'

## **Define Model**

In [29]:
from utils.models import ModelName, LLMModelName, EmbeddingModelName, get_model

model_name = ModelName.AZURE_OPENAI
llm_model, embed_model = get_model(model_name=model_name, config=config, llm_model_name=LLMModelName.GPT_35_TURBO, embedding_model_name=EmbeddingModelName.EMBEDDING_3_SMALL)

## **Load Vector Store Index**

In [30]:
from databases.vector_store import RedisIndexManager, PineconeIndexManager

redis_ojk = RedisIndexManager(index_name='ojk', embed_model=embed_model, config=config, db_id=0)
vector_store_ojk = redis_ojk.load_vector_index()

redis_bi = RedisIndexManager(index_name='bi', embed_model=embed_model, config=config, db_id=0)
vector_store_bi = redis_bi.load_vector_index()

redis_sikepo_ket = RedisIndexManager(index_name='sikepo-ketentuan-terkait', embed_model=embed_model, config=config, db_id=0)
vector_store_ket = redis_sikepo_ket.load_vector_index()

redis_sikepo_rek = RedisIndexManager(index_name='sikepo-rekam-jejak', embed_model=embed_model, config=config, db_id=0)
vector_store_rek = redis_sikepo_rek.load_vector_index()

# pinecone_ojk = PineconeIndexManager(index_name='ojk', embed_model=embed_model, config=config)
# vector_store_ojk = pinecone_ojk.load_vector_index()

# pinecone_bi = PineconeIndexManager(index_name='bi', embed_model=embed_model, config=config)
# vector_store_bi = pinecone_bi.load_vector_index()

# pinecone_ket = PineconeIndexManager(index_name='ketentuan-terkait', embed_model=embed_model, config=config)
# vector_store_ket = pinecone_ket.load_vector_index()

# pinecone_rek = PineconeIndexManager(index_name='rekam-jejak', embed_model=embed_model, config=config)
# vector_store_rek = pinecone_rek.load_vector_index()

In [31]:
from databases.neo4j_graph_store import Neo4jGraphStore

neo4j_sikepo = Neo4jGraphStore(config=config)
graph = neo4j_sikepo.get_graph()

## **Get Retriever**

In [32]:
from retriever.retriever_ojk.retriever_ojk import get_retriever_ojk
from retriever.retriever_bi.retriever_bi import get_retriever_bi
from retriever.retriever_sikepo.lotr_sikepo import lotr_sikepo

retriever_ojk = get_retriever_ojk(vector_store=vector_store_ojk, top_n=6, top_k=16, llm_model=llm_model, embed_model=embed_model, config=config)
retriever_bi = get_retriever_bi(vector_store=vector_store_bi, top_n=6, top_k=16, llm_model=llm_model, embed_model=embed_model, config=config)
retriever_sikepo_ket = lotr_sikepo(vector_store=vector_store_ket, llm_model=llm_model, embed_model=embed_model, config=config)
retriever_sikepo_rek = lotr_sikepo(vector_store=vector_store_rek, llm_model=llm_model, embed_model=embed_model, config=config)

In [8]:
# retriever_ojk.invoke("What is QRIS?")

## **Create Chain**

In [9]:
from databases.chat_store import RedisChatStore, MongoDBChatStore

chat_store = RedisChatStore(k=3, config=config, db_id=1)
# chat_store = MongoDBChatStore(config=config, k=4)

In [10]:
from chain.rag_chain import create_chain_with_chat_history, create_chain
from retriever.retriever_sikepo.graph_cypher_retriever import graph_rag_chain


graph_chain = graph_rag_chain(llm_model, llm_model, graph=graph)

chain = create_chain(
    llm_model=llm_model,
    graph_chain=graph_chain,
    retriever_ojk=retriever_ojk,
    retriever_bi=retriever_sikepo_ket,
    retriever_sikepo_ketentuan=retriever_sikepo_ket,
    retriever_sikepo_rekam=retriever_sikepo_rek,
)

chain_history = create_chain_with_chat_history(
    final_chain=chain,
    chat_store=chat_store,
)

## **Invoke**

#### **Chat Store Monitor**

In [11]:
# to print chat history
print(chat_store.get_session_history(user_id=USER_ID, conversation_id=CONVERSATION_ID))

In [39]:
# to make the chat history empty
chat_store.clear_all()

#### **With Question, Answer, and Context**

In [15]:
from chain.rag_chain import get_response

response = get_response(
    chain=chain_history,
    question="Berapa SWDKLLJ dari ambulans?",
    user_id=USER_ID,
    conversation_id=CONVERSATION_ID
)

response

{'answer': {'rewrited question': 'Berapa SWDKLLJ yang harus dibayarkan oleh ambulans?',
  'answer': 'SWDKLLJ yang harus dibayarkan oleh ambulans adalah Rp 0, karena ambulans dibebaskan dari kewajiban membayar SWDKLLJ.\n\nSource: Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan (https://www.ojk.go.id/id/regulasi/Documents/Pages/PMK-Nomor-36PMK.010-Tahun-2008-tentang-Besar-Santunan-dan-Sumbangan-Wajib-Dana-Kecelakaan-Lalu-Lintas-Jalan/menas13_1389258036.pdf#page=4)',
  'context': '[\n  {\n    "metadata": {\n      "id": "doc:ojk:661f27d6f65e4a81b951c1c0162833e3",\n      "title": "Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan",\n      "sector": "IKNB",\n      "subsector": "Asuransi",\n      "regulation_type": "Klasifikasi Bapepam",\n      "regulation_number": "36/PMK.010/2008",\n      "effective_date": "26 Februari 2008",\n      "file_

In [15]:
print(response['answer']['answer'])


Saya tidak tahu mengenai hal tersebut.


In [14]:
print(response['answer'])

KeyError: 'context'

#### **Streaming**

In [ ]:
# from chain.rag_chain import print_answer_stream

# print_answer_stream(chain=chain_history, question="Siapa itu lionel messi?", user_id=USER_ID, conversation_id=CONVERSATION_ID)

Saya minta maaf, tetapi pertanyaan yang Anda ajukan tidak terkait dengan konteks dan informasi metadata yang diberikan. Silakan ajukan pertanyaan lain yang terkait dengan kepatuhan perbankan di Indonesia.